<a href="https://colab.research.google.com/github/gsfreitas/engenharia-medica/blob/main/atividade_pratica_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

A start-up de Engenharia Biomédica na qual você trabalha é apoiada por uma grande indústria farmacêutica que pretende desenvolver novos produtos para o tratamento do Diabetes. Como fruto desta parceria, você recebeu registros de
variabilidade glicêmica de mais de 350 pacientes divididos em duas classes: pacientes com diabetes tipo 2 e indivíduos saudáveis do grupo controle. Os registros eram coletados pelos próprios pacientes em um protótipo de glicosímetro portátil automatizado e 9 biomarcadores foram extraídos a partir de tais registros. Tais biomarcadores estão disponíveis no arquivo .mat contido no material da aula 12 no Classroom *(o arquivo contém dois arrays: “Dados” = matriz com características por padrões; “Classes” = vetor com as classes de cada padrão, sendo 0 = indivíduos hígidos e 1 = pacientes com diabetes)*.

In [ ]:
# importando as bibliotecas
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as st
import itertools
import scipy.io
import scipy.fft as fft
from scipy.signal import welch
from sklearn import svm
from scipy.stats import shapiro
import seaborn as sns
import operator

In [ ]:
# leitura e tratamento dos dados
data = scipy.io.loadmat("/content/drive/MyDrive/ict-unifesp/2023.2/engenharia-medica/atividade-pratica-5/Dados.mat")['Dados']
classes = scipy.io.loadmat("/content/drive/MyDrive/ict-unifesp/2023.2/engenharia-medica/atividade-pratica-5/Dados.mat")['Classes']

In [ ]:
# criando um vetor referência unidimensional
mask = classes.ravel()

# separando as classes
higidos = data[:, mask==0]
saudaveis = data[:, mask==1]

Prepare um espaço com três características escolhidas por você visando melhor separar as classes. O código deve plotar as duas classes no espaço tridimensional escolhido. Além disso, você deverá embasar, da maneira mais completa e apropriada que puder, a escolha realizada.

Vamos realziar essa escolha baseando-se na seleção vetorial. Já que queremos obter as melhores características no espaço, então esse método é ideal, pois ele calcula a distância entre as classes para ficar mais nítida a distinção entre elas.

In [ ]:
def t3_remoutliers(padroes,p,method='desvio'):
   #Encontra outlires baseando-se em dois métodos possiveis:
    #  method = 'desvio': mediana +-p x desvio
    #  method = 'quartis': quartis  +-p x intervalo entre quartis
    # padroes = numpy array de uma característica (N x 1)
    # p = numero de desvios ou de intervalos entre quartis a ser empregado
    # retorna lista com as posicoes dos outliers no array
    if method =='desvio':
        md=np.median(padroes)
        std=np.std(padroes,ddof=1)
        th1=md+p*std
        th2=md-p*std
    elif method=='quartis':
        q3, q1 = np.percentile(padroes, [75 ,25])
        iqr=q3-q1
        th1=q3+p*iqr
        th2=q1-p*iqr
    outliers=(padroes>th1) | (padroes<th2)
    outs=[i for i, val in enumerate(outliers) if val]
    return outs

In [ ]:
# criando array com o nome dos trechos
trechos = ['higido', 'saudável']

# criando listas para armazenar os outliers de cada trecho
out_higidos = []
out_saudaveis = []

# iterando sobre cada lista para receber os outliers
for index in range(0,4):
  out_higidos.append(t3_remoutliers(higidos[:,index], 3, 'desvio'))
  out_saudaveis.append(t3_remoutliers(saudaveis[:,index], 3, 'desvio'))

# contando os valores únicos de outliers de cada trecho
outliers = [list(set(valor for sublista in out_higidos for valor in sublista)),
            list(set(valor for sublista in out_saudaveis for valor in sublista))]

# print com a quantidade de outliers de cada trecho
for index in range(len(outliers)):
  print(f"Quantidade de outliers {trechos[index]}: {len(outliers[index])}")

Quantidade de outliers higido: 1
Quantidade de outliers saudável: 1


In [ ]:
# removendo os outliers
for index in range(9):
    no_out_higidos = np.delete(higidos, out_higidos[index], axis=0)
    no_out_saudaveis = np.delete(saudaveis, out_saudaveis[index], axis=0)

IndexError: ignored